In [56]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go

In [45]:
mt5.initialize(login=1700523, server='GenialInvestimentos-PRD', password='C21j17+eh')
pd.set_option('display.max_rows', None)

In [34]:
symbol = 'WODK24'
timeframe = mt5.TIMEFRAME_M5
start_pos = 0
num_bars = 1000
volume = 1

In [61]:
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

rates = mt5.copy_rates_from_pos("WDOK24", mt5.TIMEFRAME_M5, 0, 1000)
 

mt5.shutdown()


rates_frame = pd.DataFrame(rates)
rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')

import pandas as pd

rates_frame['time'] = pd.to_datetime(rates_frame['time'])
rates_frame['date'] = rates_frame['time'].dt.date

closing_prices = rates_frame.groupby('date')['close'].last()
previous_closing_prices = closing_prices.shift(1)

date_to_previous_close = previous_closing_prices.to_dict()

rates_frame['previous_day_close'] = rates_frame['date'].map(date_to_previous_close)

rates_frame['1pct_change_up'] = rates_frame['previous_day_close'] * (1 + 0.01)
rates_frame['1pct_change_down'] = rates_frame['previous_day_close'] * ( 1 - 0.01)
# rates_frame.drop('date', axis=1, inplace=True)

rates_frame.head()

,time,open,high,low,close,tick_volume,spread,real_volume,date,previous_day_close,1pct_change_up,1pct_change_down
0,2024-04-16 11:10:00,5262.0,5264.5,5262.0,5264.0,6804,1,28253,2024-04-16,NaN,NaN,NaN
1,2024-04-16 11:15:00,5264.0,5268.0,5260.0,5267.5,12769,1,51944,2024-04-16,NaN,NaN,NaN
2,2024-04-16 11:20:00,5267.0,5274.5,5267.0,5273.0,13377,1,54775,2024-04-16,NaN,NaN,NaN
3,2024-04-16 11:25:00,5273.0,5279.5,5273.0,5278.5,11631,1,47652,2024-04-16,NaN,NaN,NaN
4,2024-04-16 11:30:00,5278.0,5282.0,5276.5,5280.0,9895,1,41919,2024-04-16,NaN,NaN,NaN


In [65]:
fig = go.Figure(data=[go.Candlestick(
    x=rates_frame['time'].dt.strftime('%Y-%m-%d %H:%M:%S'),
    open=rates_frame['open'],
    high=rates_frame['high'],
    low=rates_frame['low'],
    close=rates_frame['close']
)])

# Ajustar o tipo do eixo x para 'category' para remover os espaços em branco
fig.update_xaxes(type='category')
fig.update_layout(
    xaxis=dict(
        tickmode='auto',          # 'auto' deixa o Plotly decidir o melhor espaçamento
        nticks=20,                # Define o número máximo de ticks no eixo
        tickformat='%d %b %Y',    # Define o formato dos ticks, neste exemplo: '01 Jan 2020'
    )
)

fig.show()